In [6]:
# Imports

import requests
import urllib3

from google.cloud import storage
from zipfile import ZipFile, is_zipfile
import io

from pyspark.sql.types import StructField, IntegerType, StructType, StringType
from pyspark.sql.functions import udf, col, regexp_replace, when
from pyspark.sql import functions as F

import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
# Request of the data set

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

cookies = {
    'ASP.NET_SessionId': 'zpnwfm0oan3jqy2uwtpptpzg',
    'has_js': '1',
    '_gid': 'GA1.2.1931139333.1649668760',
    '_ga': 'GA1.3.1838557420.1649668760',
    '_gid': 'GA1.3.1931139333.1649668760',
    '__utmc': '261918792',
    'QSI_HistorySession': 'https%3A%2F%2Ftranstats.bts.gov%2FDL_SelectFields.aspx%3Fgnoyr_VQ%3DFGJ%26QO_fu146_anzr%3Db0-gvzr~1649668762469',
    'QSI_SI_e8TSnTOxZxScWdT_intercept': 'true',
    'ASPSESSIONIDCSTQDCSB': 'MHNMPDLBIKIFCHBCNNEKEGEF',
    '__utma': '261918792.1838557420.1649668760.1649668760.1649670106.2',
    '__utmz': '261918792.1649670106.2.2.utmcsr=google|utmccn=(organic)|utmcmd=organic|utmctr=(not%20provided)',
    '__utmt_GSA_CP': '1',
    'f5avraaaaaaaaaaaaaaaa_session_': 'GKOFDCDCMKLNILCIDPJGCIGFONIKKKGJGFMDKDGGMACAOADMENPMKLKAOMLNEGDENDADOCJCFLBIMIOHBPLALMJKNPCMIAENIIFEMBOHBNLNHOFLNBBMDNMIDNLHAKEI',
    '_ga_NQ5ZN114SB': 'GS1.1.1649668759.1.1.1649671209.0',
    '_ga': 'GA1.2.1838557420.1649668760',
    '_gat_gtag_UA_18660041_1': '1',
    '_gat': '1',
    '_gat_b': '1',
    '__utmb': '261918792.5.10.1649670106',
}

headers = {
    'Connection': 'keep-alive',
    'Cache-Control': 'max-age=0',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="99", "Google Chrome";v="99"',
    'sec-ch-ua-mobile': '?1',
    'sec-ch-ua-platform': '"Android"',
    'Upgrade-Insecure-Requests': '1',
    'Origin': 'https://transtats.bts.gov',
    'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.84 Mobile Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-User': '?1',
    'Sec-Fetch-Dest': 'document',
    'Referer': 'https://transtats.bts.gov/DL_SelectFields.aspx?gnoyr_VQ=FGJ&QO_fu146_anzr=b0-gvzr',
    'Accept-Language': 'fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7',
}

params = {
    'gnoyr_VQ': 'FGJ',
    'QO_fu146_anzr': 'b0-gvzr',
}

for year in [2018,2019,2021]:
    for month in range(1,13):
        data = {
            '__EVENTTARGET': '',
            '__EVENTARGUMENT': '',
            '__LASTFOCUS': '',
            '__VIEWSTATE': 'dfPCzawdgkbIZG4zgkDCOY7mHhDW5vcNkouTHUal7rIrxME2t7t0HrK9zUP6jnUFAS/FcdWgGovx9mW53hq6NpdfIIYu0u5k1XB2tVPZjucDeef8Juw9v2y4qHQYe8TBIh+LHS1wKpzQsdAJITf4KLXpIFWT82k7Hh7uW0u1GYHvEjG8R8zw5ymrTXi3yIR86fz+YDatXSnwXB/aljrpBdPQdwQZHASTuYqENtDfG/I21ZAOKo/qeQvd0VzQ4ujJycIwCJ0R54gjYo96uMPmxK5n+ru4AB8Izn+v3t0us/IV2v3pGCr5k0eokMok4uno+8fpoI+XZDeLMPycLyf4nvbNIhF1UF1LLLmdmYrm0zMbDkH78nPmWfL0c+tU62l25lXllTVF0LckaJfal+6U7bXFPGIIRAipBOjmrqqEBR0IRdSOk1goOpf2Y0eSg8xg4mMLPrXcWAd33hIfzInuo3yRddM2OpYjyBUwUEGCZSjYQn7Dkwg2IhSlCMTXLHs4lMsMaJ42eSGsg8jObz6Tbef0AwCE0opeheuQT/tQLh1VFnyu7MN0mgca2XjdMB+tpxFXfJf7j/vv+rjttSNoBmN35we0g8fXxxq624x/dpeLmEI9k9kCWbSq9aq0xnMKAIqenWBJyeBUqKRjXjcub+lM2V/yQ2xt6TXAgtBv7hqZ2wSuwjVnetzWnWP0RzfZ5pHD1W1yUF/LtfNorZUhiMjdIt48bhQW/keOQh/UJl2dTbyc8K8DK7S/SNmuYh7hr2iwvKLOs0KRVysnmmGMQFlmHiujicgt9k+g9HPdgHCYQO+ASbS3uwwWKQrGSpE20NZhp/WIxVDc4SR0WQWTxpoVgWZ5TvbZoijzRtiHBWu9uL4+Sa5nWBbVYeFBDPsits0JDiJRtQCpN3RC+HFIbloOu1J5XvUFXtTc4tavEaOMUzxYPUZydLQ084CZqZLcrD0+Z0v8EVuWFqiXLmVNatu1UVE26chHmLO/d/dYGlcygQfPHVFq+ijhE/X21WxpjI/iUGkoEw9sVVaX2KVDdVcKbYTYKRfabPGfzx82LJB/32BkTdHBX8OUtAlwYjjI0SeoUFbBvQRYGNfgTPTVmzG881TWNLWBdl5pEZTZIwjoeGRZIn7jW8hz+S5QgIRnYzMpnW3j7VRU3VEv3VqtPk1bwPe3lYwXMT80khsbUHqGHsQ3nD9ydTFvGCsHJckHSxLzIFLtq+L7s2SpnNnKnJo/Ev7+Bik4Z5QKlZE3+ri19vjE2Km1KgO6pkzbhZaLst3Ae7IAOAwqlZ7JPkVpjQ3HaSVY14ObS94kedg3Q445LSiKbtXxydu5PALLEynrYGWyb6T3wUXin4uEnCq7LbfwQlyFZbPrrbaY6UWssa2kzc4j4NXkovFCQ53W9TtNoBfWEypxprvOv9V3ORuhG3RIfzJeOdVp4ZyWkhsGMgdoKesyXgwqKgldywHRDdQ+96jQsqa3FFqYJbNyLXyPGy/zJS037RRJcIF8dCUCz2T9x37zkR39lGIuMcKJPDiSza0NAmGAgmb++bwEesSeWiMepEWX3Xh/qZBEReoSTIiTvQR+4rDh7F2FtudmO/+ZKpUn/iEHmHXtyTDRUHQeH+vRnrz9L3tRfbPEsjfF2Yyc8fu+Yg8l40kWISpgEeYIRL3ZKqV7VyW5m6yn8gl7ZRLlYop8d4S8gjA2Sd7dpGO624NqgHJG3ajkXQ6W6wNVC3Oi18SyhW44PVNp7xfnc4ghM9n7geaYuy+9u+fct9BcfoH/QjN215mi0w7ioopBAqKnWINoznQ+hCvm7qn/eTBdqethWxD9nU87Yb4T1alkcd82tjPa4v86kzJ9XB98GPvNPSTfK5Pe8WHC52A+nsn+MXGgAIZUp2jrGzwk92jMKLW6l0fxvXl+GZewPUO5DfLGbMRIG2fxR40+yYIzlu+wv67G5hw2DBd1vCLtH07D+yaHg1jMlOKwrCFtRKH0dBiU/o+x1eJrrYQdVGOEvL5+QEjM/F7tE9vAJ4JZTGVmIlnzIxp3o4GvP5doIVwLC73BRW+ZWCEoTms9t9ZhlgsoiWIHYkHWejHq63Je7Lr6mwAAFhpdn+iCCoGwjFBfufJBLPfdOFdtlvOS1db9v3hlX9IrAd+yVkSJS/jBcnhcE3reofc62j1bcH5vX1aavwy30LvqGqTONYJnCx/UKTmaD/xnl7cl/Sw9PyLy+cEKDooyanxy80oSXwkRCVn/qt1tFQLrwEEEGUshWWc8ES830yJGMG07igcCgImh5zbRhO06OKzeiWHILrk5Iiq//5WI20gu5VvRF3/zKmagHuqaE3pKR9uMSYgTKWFVkJn+2yUXygzRno6DSb9N2W4DKU3PMqhtW6VaJi8ZRVjMJyofz0abaRujZ4uYtUXRlQu7bYZYPxDu7x5pCvgkmkYdZF7Xd8ejAZYXbaWMw3JR90KQFDXll7/6J9g54DaA8cBk4Q1s4IJlKPohxQhK0nkThLcHQJcvYMq9URdjiulZmXM7dQmhPjIwCAbu6dhhEdxHAUU6QAClHh+/MiBsA+yefQ2ZIuMNKvbApp1oJOC/WL190yklj+pjFCKi/WgV1OhcS5V0cPRpAUE1PGVqcgnFONwONXUQsgIp8pZ8Ve9UMyGDMKtMQhVPW5jPYOG8GlJ/ilUtipOVr/2gS+jxsv7sYy/rM8jDuLPCYIm7aPrd0WeEIdAfHwKMgbNgoipB54H5gZ9bJdZ7k9O3/S3V60DqCe2ib6Y8nyV0+GxsYCab/4ITYZ6vx3ev9XlpO/UgrBHC0PIUiagZQXIbod6WB2xh0y9uBPBLdZoTkDmDNOYVoTX7SW1PeYsAMVN3UYBCxB9MWn+Wr0dwP8Wn4iwOWubmdWjWNnligD1hMWEMO+Sct4xL7TArniLZInsnsu941oFPlJKy/TBmOnb9Lbt/mZ3xERBZbk5Vnc0nciBHom449x4wB+qZTCDNa+sbhD+mtLaUahAXoabCxon+jYCewY8ES4LMlaDq2U5DnZDNT3NUC2mzd3kS14TBtjc/y4J1606i2v2w4erKr/pZCR8QctdAsqVd/4hdLQl5HGrXUDysMhnuNbiaXyoKn7KDUHHJbVEU28eaKaevsSX5bbAL4mGaTNese+CpcdTXMpZ+zkuzP4mdb11IhrR1ls/NxHtiqZUeBptlcGnZ8vSDW51U4V5T7VM4VyjpTeMVgJ5QPr+lwhnDdBAmQE1zQc/QbJOB53sTtBrZAP8qr+aXiCOqinQLdM2Yw2gN+pv6yRoA3Lp8+Nbi5faUpfQpjhe8KxU1N2/fmTUmMh0U6Bf64K/5HPvenQrQj9hJMREHb7DiEC/xdysqyVEKUHFrAqIohANFN9sajJ8A+O1e0tv2LWsywv58/je2whB6Zu0/9aAnddertDyhbMjxkeSSFBc2mnz7xt6GUp1pGqhBT6WbMs7hFgU+x6tpYlRcDjBgqAQgAI9hqUhTA3DBIQKbLIEl35mcxmSi/p0uzPCvVyubRzfkCFKwgfiuCm5TqrKfki3OtgwsxxOCHmBWVsYPBqRYBVdJsAbhUqN/sN4XGG/aWW1u5HqkzeuZjahnYgSsWCC6DVHEGDMo17dizJlxhf7+HmFDOck/XnjQVa1YxfZ8fr1NsoNijkwOyz5PZztr4Uc3fg36IsQZ3IU1195/FVz0UqNu57T6VVxzK9XAUUUKCDPrOP8C/nYZ7PRT1Lu3HGcEEv0pByCvXxDcSwsbJFqxIKn3CoPn0SeJsN2oru05sepiiyGn2WgPO54xb5kGscnbWlkzOcPIKzx5Nbl7hkEJlI4DAuSZnvZ0jJLmlJ6gst6l14UF4SiMy3iW6JJnEk3eb0QaRYtLC1BSbck1PsOlYgqrMB7hpvKFzYB6pa2LchWHsnLfPZ4LcOZt3jdRfhTZSB2B26NTXCu3aNF/Fsxz1L9OMfDiy1/26UPFd9PqSFC5ckozJbWi9dseTju4pstvrHvIqGz9aqCDIbuqWviucYH4ngKotFVkGu0yDiGDi1EtpYpYpWajlh60DSVH9Y7hH8HvZjJFVi67wJ5vLopa+iHk0nhBL5IudsupKnO23/+q1mtQywX7HrbxpRVT4cTzWeReJSj29/GnpYfPBeqVzLSzr9pnWyAeXU9trzD+6UNKTDwCJTvTDVXrhd17wGjZ0IUa9daMKB/heqnpe5lmsVmgmcdrsy606OVdYXkFidQ5kT3vusmcAxpcZ1QMLOtsKXeQjWygKzUXRTtBwd6UlJOmLZhlZ4e6lTLGfSuEOin6Z5bV9TBDWKI4Zz+oQWRLJsZiGKR8gjy+f3YjzSJlV8eXoVIoVza6+5crnhXrV3DLFqUdB2VIB7EuVl0Tqfw2MjUvthcKmgyv5QaWcVJZWJX+TXbxxZnO2ImVNc0xsWan+wpQxSwQt6ccX5hoSdXNto/69joFiqrSwHidNDiD+Koa/2OyipRp3BSDUMqkvpE2H/aV9jxaMDVZFeSiSbdqiAWnN6FD3B3wbd+T0Z/ASqhshgBO26wOuKiP473uecubqkzyPD7gVR+wGZfODGdDOQGaM+na6EwdpEOJQ/deTS8ZqRJJHmbwAgnQKWlrMFIJIYa+c+0Bml6UwkMiy4HzTqaiT1u6teW5Ssciqg0yf3hkDF9LMdGiKVNGvQXWd9wrYWWtp56MQhKJ8oXy+QAsLFoHchwcG69Mxgh6OV44wWIMf2XbanuU+/4o+U2B34HVioxOB2iHepG/6Wsb87bbnCwM1fjaybUminVUkw59zh0Gn7y7UiM8VGFlbQ7IpPdvQ/w6ZT0va/ALc1Nrv9EjwUMYYn7qCc2/iLDtCE1X3kXlQDw9tJHtdYhFzQIH46SV0aXOlhVQJ5n3c7T4/UaLXpQsXptx+9fsZeYQAr9pbr/zq0ei8pV6u60upXfec2u+jevvaX5mcRSNjZ4scNXHBmyzxr+9ZaoSoEylcg5obiE7kmpEIJu+Z9rQKTVn6UC471kSgLpNcybJqTiQNEppTJyPzxmblYZnDD6zBOwtdkI3HUKn9smIRWf49oZl/mYBhtqYUAUhlcTO/dILaA4VFB+klyH04YLh+wsnVDBxvmPT/toCQVOqySANpb+1/Lsvid5q+PyfDhLPGJSPngUAxq9QU+RsKpCjguk2c41124yrCt3sLoBPvMCzaqgc97RuAa4t2HNXS1iwEAca12SY/mWyurNX5hWR7N6S9K60AqalunEnvU5Lb9o63XAfHOOQg7t5Ly3YPVPfQzZ1ZZnFuwEGL9HhBbWQujuLXZlyt0SuJmXzkP0OTwHfFWUAqT4+NW62FHB3wUUZPMxqNciPlOhG53Hr4Vx9H9rO/rAflOxwKSBUl9TILU/IauqjQGR0jW4njPK8M5ftJnND/wP7LVpetqUr4d/M09By+vm4pp1qrJ3SRxWqgh61dXIfPqo/tX3ZJYXAoqktGTXsPy3Z+JnknM6lniajsffGj05Sv0QGl1uGzE4PCcsORjUfKL7iYtnj5VDj8LBOKpdIfFau724xPdmv9vM95Sb7t2eTEPVFCA5ijlyyb/XhW3A8OEcDfI9lwL9yd/yD02Nv7kunW9fymhatkRYioPRSJO6x3xIecl5/d7rw88JgZkk2yoSqODk+UA77GoiEq2f1Yq64IzA885b+ykJgWGypNSN2QwiFHrDEWSA0HzcAng56PNy1hkhfVEDzgviWHx7DAxzALkCrBh3DqiBSep+d3HhqjuhVHbcIZrf3E1QGAgiEudY9N2hZMsUnWo4GphWBvLyxAftEL+VFp4rh33tuGeUqe6a3830Hfby9cgjMOrIGTIt2jtxHo736m5MEEPYSLw+LShfmRl6bAHRgLx8h6wnnCuvHl+KMmLzzaLzbkyJ5On1Wl3IgnW2+JQOAVPTae9hGtD4SH4qs8mb6neHNBvcwzFjsQceHwYjBl03mcQD0LP7qkzUdZRwmy2pjG2Op+ODgDA0q4zkAgDR80RE0eEPkBxSwQS/FfMfvxfTZ0K8mSh3VMmq8UqXw3eJg+WVActVFO1KszYoG6SXQjTxxQtLpLLCK01qxMHiIUdZ5NfBydgFymcJh1UONFxngSaGmugps7JStlLC3AdmQLmI9l1zzxt21ULS0FJezKKMj4h6tkAwXkGHn7emaQ0LvHatx6ByBUxQiprGBUOWUZIivj/gv62d4oldUw6v4+iLLLrAvtwCWejA08AUz+15/5mNdZ/Gu2HV/JTPMyfE3o11FeDAzRcnxAkk+D/ilnlWrbSD3N6TLWhAo+q0cdyCuAjnhdNAK/nyP8OQdYzzG6935aLxUkm5/x2MXqTol92jJlbHzICGmPBDn2QdqbTQdlYDXFnrUs3qR+bVgw129gJor1RaxFswPmlPih450WcKsvhXz25GucBGwsDnr+BiDCmisuG8RtpuzTEB5Xwquanqt7vQZCNG4E5nPu5E2pOSsFXOimGuRlzLLhTTi6s2n5eFFlHVsrgOTMg==',
            '__VIEWSTATEGENERATOR': '82CE8F70',
            '__EVENTVALIDATION': '/fK6wzbBlZ/aJt4M0QYPGE9sRwh5Sk6U4vUZRxqjrBs+Tf0hHt2RlQxNxShxJViqlWpPQu9kuqi8DQ3g29CtVvIxnMvgQI9X6x7E+ODLAAgHETrNrAxTsklytqH2Xn/H4oMwkeg5UJxvgaPTyY2ngis+EE0k5W7vbKfe7OupRly5+XEqoAqrKkN2oJF58owBFKXYub/MMcDHABj5ou0Ae7KbsySywL0CXNlqZu+cllwRmtpgySAj+V3t5wJ4kC3/kQVsR3EUxSDOVnOyecQlT/vPj5p4HO0xX9hdsxx27EtMgE3xrri+N7ba6ub/vn/hUyPr5sTxjvZlSN/pLpc51JScvH2oytjU7JGXuN7Shdi+HGE/vRrU0HPQh6ZCUJrsikwkpH9/Yu/hiNVxeJGuhgz6neXLBNsu9lJ5RIGbiplWUotP8Gu+Ld//St5JnxXkBfxal1sIUi7i6gzKIbDH6+EdPf6Bv8LIwad41HHkopgy0EsVasX4YtmI0egoDQEpfmtEMFqpmIfltBc5llMdzmXXqaEPVxAXhXhB+VRFKlNrFJRJl+aoaqG8CKG0sr1vnNuwfb2xudsFK6o4ZUAY0Hrzuq9R2OZFBNBSNpRMSg9YXV8+3vameqth9ZnS/1Zb5M8/IVi2xIa7gpol/qF0enfJh5xFZZqdRlWCIE4G5eVLtPPRDNixFjWdMXxwd3blXzktwi/yR21NKbsLtkCkZGdHgZHJFxD8+zxP4pscE35ZOLU8kwuYkkYlR7TBp/CSXlgLp4kLekJY4bw8I/dlxAt7iVPcCfXfpr/GwPUXZrjEw5S4yKpM2q/qjPF5s5tnkccZlQotnhuodnYDvFPRBgzYLBsc/GpkCUd1gDeOTNwj4Agjd1n/3WSMD6qSPmN4z/wSpEL+U0pSJJOghbhREQHrgPjQuV6NmcX68IMknee1AqlStgV3CjbUDC4EHtFjXvFNLA+zqE6jDfRNOcWzVr2HWzRflh9hDJP3IfekaHg24dC94cBiWSEfluUUiiHY/fSCnmZf51kFX+PbsN8g83YxG3vTR89OaZIb5ARdP14ktpaWz0XrJMnnj2gkFfGyCIrPcSM/YmKBH6T2QgqibEb5qX2YiOr+VDymvNkIMGHAfKCDdjZUU/17udlGYlkOJIm0ULM1hQxKtNLyZsXis5XwWq+XiYTvXrfeqV+sESrbOXrqfklBM1oKsS3NoG3diBd7phnQzmafgzaiZlKW+sq0WK7Z66C3QRCz69RXk86zalrMZEiDz6Zrqpj73sPl4MrDbHwGih1HvbVwjSuGILbEpZ7Ew1NwVroLfXfDFgKvkxxZGiO1Up3pCyckOn3mUJmfu9czoBqrqZQx5X/HFS3/nqFuiv0ufrOBao7Z1rYN1zrQag1ma9WasaQhFoRhAh6fuEAW2TzI3rmbFe+Y86rYH4rv0yESKQkRhHwzZn6hFqd9zO8fUfo+KMN2aIRFHI7UHjsqGz+FvODaSRRWjZcj0crTDg4R5UlugshBPncA5fYT0uWagx1kOp6GPLcExPJOtUG8pgvoboYksWp787k0Q/jLaas82pxLyYSpkUJs9ZSOGanYxmbqDpH4UuIu31gNuxMPtmaCYVjIybEirGPkVCR7NhIz25nR917qff/NZALcnUB/VYxLRXsZ/r1VV49ztEuQk3qniOdp2hJPGmVql5inFQhYnU5Z+Zn5khWs7svnIda0/E9OSMApTxkef7kMiQ+80qc8Wr4yWeFDCrnYxmkl4VJJSSWeWrU42tpSwOKczeZIW93kA6NOZuaRrrSg36ggUdunkjwfcbNS9nYdOmHGj9XbNdaKwEDxv7tpFCR+lygiAveDl6o02GDMJ7ABQfbW7LtxCK5hOp78QLSvhcbbOqqNfh+cda7h/eefju82CRbWrwwu+CGBS98NJ1AEdBiKdN8WvqCuLwA1AckhRZIyBSPuP3aamaH9vvF2rrdXXwTNy5MTjTclTzrmo/tIF7eGG+R5hOSXwgVndvrXL+Eq8Lrx6yfLWJnXYgHG8gkAJfm/piSGSJkfj2UcKAaGybnJyqE25QRRmUST3iSNdfRFhH/rgG1AAjlMQW5nK8UfUNlrla22Fz9v3JFbsSWjB0iBBp3oEF+cQ93SJwB0ZgGHURh0GosD//J57zW7OIy+MzTuX/Td+zP90as2QWd2RG5E0HhqCfE/HLqkTZ8naMMRuY+E5lxwHiBi3Pv7b3qBg5P6bzySqyArXzp4Vj4cfLHS/faaPosUXk5JUwaA8caCtGSk1Ja8TumfO+sRzY5GGbP/zjfp6U1WhCkybLg+W1Xse4+PY1u9einOnBm9fF1Vqa7yrr++lqN2E2GpG42FAIyVLx+K7SR6qtLZACBwso9+74yHISVFJYds7sIt8RqtO+P5BQpzlGhwaC1jyLfEss3zlQl+L/rcTlowAu1MAYDT2jZWymCIlrrA74VzdblBKT+1c5mC7dhdg49DXo2IzU1eovZcAryp5KvzNTWisd6if8vOhqo+rVTuT88nueqWkUE4GnGqhCTOeKYaj8SpapL7wCB/I4soRIyBs1HWWU6v3957DyA/eYuIi1H4Rjy/H7x1QvaICDvZFKx681fd1FkD45En9bRAjRJKCq3ObytpKZYb40TDasbtKOR8aeo2Xxo0kk8CPkIeNqZGtzPGyNFnR77DZ6qKZauZGe9zXv1Aw8/1xb4kgT/VZgnHx7VgaaOkd5VUMlPan+i+ZqOEZlVmltMxubFm+eyESvq87AsntI2/Is7EcX3A6SX/JWLm3On7AkSIjngpiXg4slWel4G/ThQXAOc2Am/yC508rZeEiWu5ZB9n+CrokUXZ6zC3pRG9KOhSLyVgIkfRYy7OXMxoTdky71IzWLnPFTKvIeM9AhdSCKjrWnWuOS56hRnTJ04US+q/yOzvai0lQMlRg5a2lsXR3ezdQeXexx0Mz0TR9QvNjPmPVI6U3ihKO4wEh+RRrOY2ZLPeR321HTclsg/Ygs7BJPjStTecNaBCdy5HTxO7Vnwd5XUwIbC+5IstWhl5yTlPF/P+KxswTtb1v04u8G05692w2R2wHpKI0jjza6gZCWSFuDs7Wi1GYybf2+RHHV4FDyKX6mU45YJFamvTnHGwtX2U8/6LBxXUKkEn1JOnnWcF4S1v3p6+dL0/yW7+Y63nlhxoMGy7Rp8awjoNeshrGujbYDEV1CL/DMUyx4kM8/oy4UFt4MlbfqVkdkk66kmff9Ss0wxOEqFuAj/ZCvYk8taYn+oJDcpja8QBXaeGKlDPehR7F/12FpZXxLqMPf8NN7x6jglnGLFMPeoIlYlA71avuUEH1fe8+ZKCIOsIGRnOnvJZ4xtZNAr8y+ZUbuftatQkvdEL3tMOc8O7+svbEXr8tM3GSWybHDe9yeU6HIDmQxcjvCnNOjaox6mpDNLzYEwp7s5r5E4dqc06XVtiNNYYE6JjGLmLa7Yr1934NqGmFD5/RPDdtXfgwwTAhS7U/wj9jVg3H+PJlZODAT/6JbtIXg0Jfz/NR/NE+o21XTx2b40Nnj98FRvF3b0fqXKejE7TD7wuw6xuxp4Zm4gcF755GmC9TSX0o9E5O3ALqC/6ejRFg4s4SNgCA2bW2nqIFAKETCiOyeh6/q5Gk3sxxt+Kgr0q5PIwFGbyRRUFs/sDMYOX57zTzIC/yYkJr690IcMMGo3Wb88utqyOMzf/gl0FbBbGrtxSxjAn+9a6+DEHRq7TN9bJhjcLlTpZDmu/hJGR/CWfP/DpCexgY/PYM4zWmQompwZxlXBXwiopDpn73SfyUMnDvLCXbkHO3j3ImrsAmZ9IgTedkomiAwBaU2jmjVEBn+c9RBHmbS0AJMHwwBBL1Q9IqOJ1mnMnC7W7GkRG4vFy6fPPMYLS83aDpvQDFKqAwXUNupIaamPaGWkDH6RSKIGMeX0vkzDBspZJS1ExE3XV+jjUqYV35lgb1HTnLU7QWXS2n7JwYVeOy49BSmoCH6wHBePdnxZYwH6DdeKRhE4TlZrXqnOHH99A5bmciWK0l9FSecXB1CzOmK3aTkjd4fKPzEJFup10w2WgIznXON6O82i3mhlQSKcNdrPpRxNMFIQPRGF4s6MDg5K7AHIlQP+1lXviOTVCyiDoaSYQMPrLLTNJ64SwM+ZRTJnsKIO41Yz3EPZvYoPNrSP1ovVUwCn0+xkWnAPf+eTbKZsRVlSuRNXlGbu0Foq+WUqtlGDdN+KS8AIbHnYjDXzecn+kXYdkpotP44yPCvsOosssgnUfCBosDahpNXEU/QrQ3OAPh6QTaMeIyyc3lnmHQBIqRLbkFT0zw57yDqHVnGMvVQ3TS6WMupN/mR35R45Bypk8HLX5c45auiLuA5mY2gQvk9YIM3EUzqFq1M96KP035gf/R/RTnIE/MkBxxq205FZegSpPU7T5gwqW0rhwNEJWNYfZzAsGxIqGJbj3DnM/QY1+Za6qi1CxUlWBe01B0TwaejcXMZxX+yhl1ybkx5c+VKJf2dkN8WSM8b33qiynxS330R4SR4in+hty8iWK79FisLL3gPJPSp/WkVJFmJ9jMrJB15q5x9GkhofuSy8BIIzb5GaxTGBtq9hZ7KldzK/n4ld3qxYa0v3k4B/RlH4myLYQDph39mo0YLabg0+7Q5dbPZ9rXrlqdB2LquGLrMH+ExLRcUUpSYDHHmH/7lcRKyEyAayM8EzmRy1Xlq1jsmMjSPlIsYhvnJGD+o0DwDA5q96PRR6u4FnuoMs1lbe3Ug==',
            'txtSearch': '',
            'cboGeography': 'All',
            'cboYear': year,
            'cboPeriod': month,
            'btnDownload': 'Download',
            'DAY_OF_WEEK': 'on',
            'OP_UNIQUE_CARRIER': 'on',
            'ORIGIN': 'on',
            'DEST': 'on',
            'DISTANCE': 'on',
            'ARR_TIME_BLK': 'on',
            'DEP_DELAY_NEW': 'on',
            'ARR_DELAY_NEW': 'on',
            'DEP_TIME_BLK': 'on',
            'YEAR': 'on',
            'MONTH': 'on'
        }
        response = requests.post('https://transtats.bts.gov/DL_SelectFields.aspx', headers=headers, params=params, cookies=cookies, data=data, verify=False)
        data = response.content
        f = open('/Plane_data/'+str(year)+'-'+str(month)+'.zip', 'wb')
        f.write(data)
        f.close()

FileNotFoundError: [Errno 2] No such file or directory: '/Plane_data/2018-1.zip'

In [18]:
# Copying the requested data in our working directory

!gsutil -m cp Plane_data/*.zip gs://dataproc-staging-us-central1-881288578123-co6sg2sg/Plane_data/

Copying file://Plane_data/2018-1.zip [Content-Type=application/zip]...
Copying file://Plane_data/2018-10.zip [Content-Type=application/zip]...
Copying file://Plane_data/2018-11.zip [Content-Type=application/zip]...         
Copying file://Plane_data/2018-12.zip [Content-Type=application/zip]...         
Copying file://Plane_data/2018-5.zip [Content-Type=application/zip]...          
Copying file://Plane_data/2018-3.zip [Content-Type=application/zip]...          
Copying file://Plane_data/2019-1.zip [Content-Type=application/zip]...
Copying file://Plane_data/2018-4.zip [Content-Type=application/zip]...
Copying file://Plane_data/2018-6.zip [Content-Type=application/zip]...          
Copying file://Plane_data/2019-5.zip [Content-Type=application/zip]...          
Copying file://Plane_data/2019-12.zip [Content-Type=application/zip]...         
Copying file://Plane_data/2018-8.zip [Content-Type=application/zip]...          
Copying file://Plane_data/2019-3.zip [Content-Type=application/zip]

In [55]:
# Unzip our data and format it in .csv files

def zipextract(bucketname, zipfilename_with_path):

    for year in [2018,2019,2021]:
        for month in range(1,13):
            storage_client = storage.Client()
            bucket = storage_client.get_bucket(bucketname)

            destination_blob_pathname = zipfilename_with_path+str(year)+'-'+str(month)+'.zip'

            blob = bucket.blob(destination_blob_pathname)
            zipbytes = io.BytesIO(blob.download_as_string())

            if is_zipfile(zipbytes):
                with ZipFile(zipbytes, 'r') as myzip:
                    for contentfilename in myzip.namelist():
                        contentfile = myzip.read(contentfilename)
                        destination = str(year)+'-'+str(month)
                        blob = bucket.blob('Plane_csv/'+destination+'.csv')
                        blob.upload_from_string(contentfile)

zipextract('dataproc-staging-us-central1-881288578123-co6sg2sg', 'Plane_data/')

In [2]:
# Spark can aggregate all .csv in a single dataframe

df = spark.read.csv('gs://dataproc-staging-us-central1-881288578123-co6sg2sg/Plane_csv/', header=False)
df = df.toDF('YEAR','MONTH','DAY_OF_WEEK','OP_UNIQUE_CARRIER','ORIGIN','DEST','DEP_DELAY_NEW','DEP_TIME_BLK','ARR_DELAY_NEW','ARR_TIME_BLK','DISTANCE')

In [ ]:
df

In [ ]:
df.printSchema()

In [3]:
# This fix an issue where features name appeared as the first row of our dataframe

df = df.where(df.YEAR != 'YEAR')

In [ ]:
# First statistical analysis

df.describe().show()

In [ ]:
# We decided to remove the dupes to avoid distorting our dataset

clear_df = df.drop_duplicates()

In [ ]:
# Statistical analysis to see what changed

clear_df.describe().show()

In [8]:
df.select("OP_UNIQUE_CARRIER").distinct().show()

+-----------------+
|OP_UNIQUE_CARRIER|
+-----------------+
|               UA|
|               NK|
|               AA|
|               EV|
|               B6|
|OP_UNIQUE_CARRIER|
|               DL|
|               OO|
|               F9|
|               YV|
|               MQ|
|               OH|
|               HA|
|               G4|
|               YX|
|               AS|
|               QX|
|               VX|
|               WN|
|               9E|
+-----------------+



In [4]:
# Reformating the OP_UNIQUE_CARRIER feature column to be more intelligible

df = df.na.replace({ 
            'OO': 'Skywest Airlines', 
            'QX': 'Horizon Air', 
            'UA': 'United Airlines' , 
            'WN': 'Southwest Airlines', 
            'DL': 'Delta Airlines', 
            'F9': 'Frontier Airlines', 
            'G4': 'Allegiant Air', 
            'HA': 'Hawaiian Airlines', 
            'MQ': 'Envoy Air', 
            'NK': 'Spirit Airlines', 
            'OH': 'Jetstream Intl', 
            'AA': 'American Airlines', 
            'AS': 'Alaska Airlines', 
            'B6': 'Jetblue Airways', 
            'YV': 'Mesa Airlines', 
            'YX': 'Republic Airlines', 
            '9E': 'Endeavor Air',
            'EV': 'ExpressJet',
            'VX': 'Virgin America'},'OP_UNIQUE_CARRIER')

/usr/lib/spark/python/pyspark/sql/dataframe.py:2213: UserWarning: to_replace is a dict and value is not None. value will be ignored.
  warnings.warn("to_replace is a dict and value is not None. value will be ignored.")


In [7]:
# DAY_OF_WEEK was in an unusable format so we cast it to string
# Reformating the DAY_OF_WEEK feature column to be more intelligible

df = df.withColumn('DAY_OF_WEEK', col('DAY_OF_WEEK').cast(StringType()))
df = df.replace({ 
            '1': 'Monday', 
            '2': 'Tuesday', 
            '3': 'Wednesday', 
            '4': 'Thursday', 
            '5': 'Friday', 
            '6': 'Saturday', 
            '7': 'Sunday'
        }, 'DAY_OF_WEEK')

/usr/lib/spark/python/pyspark/sql/dataframe.py:2213: UserWarning: to_replace is a dict and value is not None. value will be ignored.
  warnings.warn("to_replace is a dict and value is not None. value will be ignored.")


In [8]:
# Reformating the MONTH feature column to be more intelligible and fixing an issue where our months values where changed with days values

df = df.withColumn('MONTH', regexp_replace('MONTH', 'Monday', 'January'))
df = df.withColumn('MONTH', regexp_replace('MONTH', 'Tuesday', 'February'))
df = df.withColumn('MONTH', regexp_replace('MONTH', 'Wednesday', 'March'))
df = df.withColumn('MONTH', regexp_replace('MONTH', 'Thursday', 'April'))
df = df.withColumn('MONTH', regexp_replace('MONTH', 'Friday', 'Mai'))
df = df.withColumn('MONTH', regexp_replace('MONTH', 'Saturday', 'Juin'))
df = df.withColumn('MONTH', regexp_replace('MONTH', 'Sunday', 'July'))
df = df.withColumn('MONTH', regexp_replace('MONTH', '8', 'August'))
df = df.withColumn('MONTH', regexp_replace('MONTH', '9', 'September'))
df = df.withColumn('MONTH', regexp_replace('MONTH', '10', 'October'))
df = df.withColumn('MONTH', regexp_replace('MONTH', '11', 'November'))
df = df.withColumn('MONTH', regexp_replace('MONTH', '12', 'December'))

df = df.withColumn('DEP_DELAY_NEW', when(df.DEP_DELAY_NEW > 0, 'True').otherwise('False'))
df = df.withColumn('ARR_DELAY_NEW', when(df.ARR_DELAY_NEW > 0, 'True').otherwise('False'))

df.show()

+----+-----+-----------+-----------------+------+----+-------------+------------+-------------+------------+--------+
|YEAR|MONTH|DAY_OF_WEEK|OP_UNIQUE_CARRIER|ORIGIN|DEST|DEP_DELAY_NEW|DEP_TIME_BLK|ARR_DELAY_NEW|ARR_TIME_BLK|DISTANCE|
+----+-----+-----------+-----------------+------+----+-------------+------------+-------------+------------+--------+
|2019| July|     Monday|  Spirit Airlines|   DEN| MCO|         True|   0700-0759|        False|   1200-1259| 1546.00|
|2019| July|    Tuesday|  Spirit Airlines|   DEN| MCO|        False|   0700-0759|        False|   1200-1259| 1546.00|
|2019| July|  Wednesday|  Spirit Airlines|   DEN| MCO|        False|   0700-0759|        False|   1200-1259| 1546.00|
|2019| July|   Thursday|  Spirit Airlines|   DEN| MCO|        False|   0700-0759|        False|   1200-1259| 1546.00|
|2019| July|     Friday|  Spirit Airlines|   DEN| MCO|        False|   0700-0759|        False|   1200-1259| 1546.00|
|2019| July|   Saturday|  Spirit Airlines|   DEN| MCO|  

In [9]:
# DEP_TIME_BLK and ARR_TIME_BLK were in an unusable format so we cast them to int

df = df.withColumn("DEP_TIME_BLK", F.regexp_extract(F.col('DEP_TIME_BLK'), '^\d\d', 0).cast("Integer"))
df = df.withColumn("ARR_TIME_BLK", F.regexp_extract(F.col('ARR_TIME_BLK'), '^\d\d', 0).cast("Integer"))

In [10]:
# We decided to change hours values to a categorical feature to make it easier to use for our model

def what_moment(x):
    #x = int(x)
    if 0<=x<6 :
        return('Night')
    elif 6<=x<12 :
        return('Morning')
    elif 12<=x<18:
        return('Afternoon')
    else:
        return('Evening')
    
what_momentUDF = udf(lambda z: what_moment(z), StringType())
spark.udf.register("what_momentUDF", what_momentUDF)

<function __main__.<lambda>(z)>

In [11]:
df = df.withColumn('DEP_TIME', what_momentUDF(F.col('DEP_TIME_BLK')))
df = df.withColumn('ARR_TIME', what_momentUDF(F.col('ARR_TIME_BLK')))

df = df.drop('YEAR','DEP_TIME_BLK','ARR_TIME_BLK')

df = df.withColumn('DISTANCE',df.DISTANCE.cast('int'))
df.show()

+-----+-----------+-----------------+------+----+-------------+-------------+--------+---------+---------+
|MONTH|DAY_OF_WEEK|OP_UNIQUE_CARRIER|ORIGIN|DEST|DEP_DELAY_NEW|ARR_DELAY_NEW|DISTANCE| DEP_TIME| ARR_TIME|
+-----+-----------+-----------------+------+----+-------------+-------------+--------+---------+---------+
| July|     Monday|  Spirit Airlines|   DEN| MCO|         True|        False|    1546|  Morning|Afternoon|
| July|    Tuesday|  Spirit Airlines|   DEN| MCO|        False|        False|    1546|  Morning|Afternoon|
| July|  Wednesday|  Spirit Airlines|   DEN| MCO|        False|        False|    1546|  Morning|Afternoon|
| July|   Thursday|  Spirit Airlines|   DEN| MCO|        False|        False|    1546|  Morning|Afternoon|
| July|     Friday|  Spirit Airlines|   DEN| MCO|        False|        False|    1546|  Morning|Afternoon|
| July|   Saturday|  Spirit Airlines|   DEN| MCO|        False|        False|    1546|  Morning|Afternoon|
| July|     Sunday|  Spirit Airlines|

In [14]:
# Saving our dataset in another .csv file to so we can reuse it later

df.write.option('header', True).mode('Overwrite').csv('gs://dataproc-staging-us-central1-881288578123-co6sg2sg/clear_csv')